<a href="https://colab.research.google.com/github/angelaapp28/fall25/blob/main/Copy_of_DeepLearning_Fall2025_hw4_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning (Fall 2025) - Homework 4

---



*Developed by Hongtau Wu & Suzanna Sia*

This notebook contains all starter code for Homework 4. Please read the written assignment carefully to ensure you include all necessary outputs in your final report. Your final submission (a single zip file) should include this notebook (.ipynb file) and a PDF of this notebook with all cell outputs.

## Problem 1a)

### Imports

In [ ]:
## External Libararies
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

### Data Loading

In [ ]:
## Spectify Path to Provided Data Here
DATA_PATH = '/content/DeepLearning_Fall2025_hw4_prob1_data (1).npy'

## Load Data and Check Dimensionality
data = np.load(DATA_PATH)
Y = data[:,2]
X = data[:,0:2]
print("Y:", Y.shape)
print("X:", X.shape)

## Polygon Boundaries
p = [[[500, 1000], [300, 800], [400, 600], [600, 600], [700, 800]],
     [[500, 600], [100, 400], [300, 200], [700, 200], [900, 400]]]
p = np.asarray(p)
p0 = p[0]
p1 = p[1]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Visualization Code

Do not touch any of the visualization code below.

In [ ]:
## Helper code for visualisation (No Need to Touch)
def visualize_polygons(p0, p1):
    """

    """
    fig, ax = plt.subplots()
    patches = []
    polygon1 = Polygon(p0, closed=True)
    polygon2 = Polygon(p1, closed=True)
    patches.append(polygon1)
    patches.append(polygon2)
    p = PatchCollection(patches, cmap=matplotlib.cm.jet, alpha=0.4)
    ax.add_collection(p)
    ax.autoscale_view()
    plt.show()

def visualize_datapoints(X, Y):
    """

    """
    assert(X.shape[0] == Y.shape[0])
    fig, ax = plt.subplots()
    npts = 60000
    col = np.where(Y[:npts]==1,'m','b')
    x1 = X[:npts][:,0]
    x2 = X[:npts][:,1]
    ax.scatter(x1, x2, s=0.5, c=col, zorder=1)
    plt.show()

In [ ]:
visualize_polygons(p0,p1)

### Problem 1a)

Please fill in all code blocks marked with a #TODO.

In [ ]:
def threshold_activation1(x):
    """

    """
    y = np.copy(x)
    y[y <= 0] = 0
    y[y > 0] = 1
    return y


def and_gate(x):
    """
        x: np array of shape (n, 1)
        return: 1 if all elements of x are 1, else 0
    """
    for i in x:
        if i <= 0:
            return 0

    return 1


def or_gate(x):
    """
        x: tuple value from AND gates
    """
    if x[0] <= 0 and x[1] <= 0:
        return 0
    else:
        return 1


def analytical_parameters(p0, p1):
    """

    """
    ## Dimensionality
    x_dim = 2
    class_num = 2
    hidden_unit_num = 10
    # First Layer Parameter
    W = np.zeros((hidden_unit_num, x_dim))
    b = np.zeros((hidden_unit_num, 1))
    for i in range(5):
        # First polygon
        x1 = p0[i, 0]
        y1 = p0[i, 1]
        x2 = p0[(i+1)%5, 0]
        y2 = p0[(i+1)%5, 1]
        W[i, :] = [y1 - y2, x2 - x1]
        b[i, :] = x1 * y2 - x2 * y1
        # Second polygon
        x1 = p1[i, 0]
        y1 = p1[i, 1]
        x2 = p1[(i+1)%5, 0]
        y2 = p1[(i+1)%5, 1]
        W[i + 5, :] = [y1 - y2, x2 - x1]
        b[i + 5, :] = x1 * y2 - x2 * y1
    return W,b

def predict_output_v1(X, W, b):
    """

    """
    predictions = []
    for idx in range(data.shape[0]):
        x = np.reshape(X[idx, :], (2, 1))
        # First layer
        first_layer_output = np.matmul(W, x) + b
        first_layer_output = threshold_activation1(first_layer_output)
        # Second layer
        first_polygon = first_layer_output[0:5, :]
        second_polygon = first_layer_output[5:10, :]
        first_gate_output = and_gate(first_polygon)
        second_gate_output = and_gate(second_polygon)
        # Output layer
        input_to_final_gate = [first_gate_output, second_gate_output]
        prediction = or_gate(input_to_final_gate)
        predictions.append(prediction)
    return predictions

def predict_output_v2(X, W, b):
    """
    """
    ## Cache of Predictions
    predictions = []
    ## Cycle Through Data Points
    for idx in range(data.shape[0]):
        x = np.reshape(X[idx, :], (2, 1))
        # First layer
        first_layer_output = np.matmul(W, x) + b
        first_layer_output = threshold_activation1(first_layer_output)
        # Second layer
        first_polygon = first_layer_output[0:5, :]
        second_polygon = first_layer_output[5:10, :]
        first_gate_output = and_gate(first_polygon)
        second_gate_output = and_gate(second_polygon)
        # Output layer
        input_to_final_gate = [first_gate_output, second_gate_output]
        prediction = first_gate_output
        predictions.append(prediction)
    return predictions

def calc_accuracy(true_y, pred_y):
    """

    """
    true_prediction_num = 0
    for i, py in enumerate(pred_y):
        if py == true_y[i]:
            true_prediction_num += 1
    accuracy = true_prediction_num / len(pred_y)
    print("Accuracy: ", accuracy)
    return accuracy

*Sanity check:* If you correctly implemented the 'and gate' and 'or gate', all points should be classified correctly when you make predictions using `predict_output_v1()`

In [ ]:
## Load Our Parameters
W, b = analytical_parameters(p0, p1)

## Make Predictions
pred_Y = predict_output_v1(X, W, b)

## Compute Accuracy
acc = calc_accuracy(Y, pred_Y)
assert (acc == 1)

## Visualize Predictions
visualize_datapoints(X, np.array(pred_Y))

In the code above, change the gates in `predict_output_v2()` such that only the points in the top polygon are classified correctly. Visualize your result, report the accuracy of this model, and attach it to the submission.

To further clarify, you should **only** change the usage of the gating functions, not the code inside the gating function itself.

In [ ]:
## Load Our Parameters
W, b = analytical_parameters(p0, p1)

## Make Predictions
pred_Y = predict_output_v2(X, W, b)

## Visualize Predictions
visualize_datapoints(X, np.array(pred_Y))

## Problem 1b-d)

Complete problems 1b through 1d in the space below. Please use markdown to clearly distinguish your answers for each part. Include appropriate visualizations generated here in your final report.

Problem 1.b

In [ ]:
import torch
import torch.nn as nn

# problem 1.b


class MLP1(nn.Module):
  def __init__(self):
    super().__init__()

    self.first_layer = nn.Linear(2, 10)
    self.second_layer = nn.Linear(10, 2)
    self.output = nn.Linear(2, 1)
    self.activation = nn.Sigmoid()

    for layer in [self.first_layer, self.second_layer, self.output]:
      nn.init.xavier_uniform_(layer.weight)

  def forward(self, x):
    x = self.activation(self.first_layer(x))
    x = self.activation(self.second_layer(x))
    x = self.activation(self.output(x))

    return x


In [ ]:
def accuracy(y_hat, y):
  prediction = (y_hat > 0.5).float()
  return (prediction == y).float().mean().item()

In [ ]:
import numpy as np
import torch
import torch.nn as nn

def train(model, x, y, test_x, test_y, loss_function, accuracy_function, batch_size = 128, max_epochs = 500, lr = 0.01):
  loss_function = nn.BCELoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=lr)

  if isinstance(x, np.ndarray):
    x = torch.FloatTensor(x)

  if isinstance(y, np.ndarray):
    y = torch.FloatTensor(y)

  if isinstance(test_x, np.ndarray):
    test_x = torch.FloatTensor(test_x)

  if isinstance(test_y, np.ndarray):
    test_y = torch.FloatTensor(test_y)

  if batch_size is None:
    batch_size = len(x)

  dataset = torch.utils.data.TensorDataset(x, y)
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

  losses = []
  accuracies = []
  test_accuracy = []

  for epoch in range(max_epochs):
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for batch_x, batch_y in dataloader:

      y_hat = model(batch_x)
      loss = loss_function(y_hat, batch_y)
      acc = accuracy_function(y_hat, batch_y)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()
      epoch_acc += acc

    accuracies.append(epoch_acc / len(dataloader))
    losses.append(epoch_loss / len(dataloader))


    avg_loss = epoch_loss / len(dataloader)
    avg_acc = epoch_acc / len(dataloader)

    # evaluation

    model.eval()
    with torch.no_grad():
      test_prediction = (model(test_x) > 0.5).float()
      test_acc = (test_prediction.eq(test_y).sum().item()) / len(test_y)
      test_accuracy.append(test_acc)


    if epoch % 100 == 0 or epoch == max_epochs - 1:

      print("iteration",epoch,"loss:", avg_loss, "accuracy:",avg_acc)



  fig0=plt.figure(0)
  plt.plot(losses,'-')
  plt.xlabel('Iteration', fontsize=18)
  plt.ylabel('Loss', fontsize=16)
  plt.show()
  fig1=plt.figure(1)
  plt.plot(accuracies,'-')
  plt.xlabel('Iteration', fontsize=18)
  plt.ylabel('Accuracy', fontsize=16)
  plt.show()

  return losses, accuracies, test_accuracy

In [ ]:
mean = X.mean(axis = 0)
std = X.std(axis = 0)
x_norm = (X - mean) / std


x_train, x_test = x_norm[:50000], x_norm[50000:]
y_train, y_test = Y[:50000], Y[50000:]


X_train = torch.tensor(x_train, dtype=torch.float32)
Y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(x_test, dtype=torch.float32)
Y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

seeds = [0, 1, 2, 3, 4]
results = []


train_accuracy, test_accuracy = [], []

for seed in seeds:
  torch.manual_seed(seed)
  model = MLP1()

  losses, train_accs, test_accs = train(
        model,
        X_train, Y_train,
        X_test, Y_test,
        nn.BCELoss(),
        accuracy,
        batch_size=128,
        max_epochs=500,
        lr=0.01
    )
  results.append((losses, train_accs, test_accs))


final_train_accs = [r[1][-1] for r in results]
final_test_accs = [r[2][-1] for r in results]

print("\n" + "="*60)
print("FINAL RESULTS - Problem 1c")
print("="*60)
print(f"Mean Train Accuracy: {np.mean(final_train_accs):.4f} ± {np.std(final_train_accs):.4f}")
print(f"Mean Test Accuracy:  {np.mean(final_test_accs):.4f} ± {np.std(final_test_accs):.4f}")
print("="*60)

print("\nIndividual Seed Results:")
for i, (train_acc, test_acc) in enumerate(zip(final_train_accs, final_test_accs)):
    print(f"Seed {i}: Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}")


with torch.no_grad():
    preds = (model(torch.FloatTensor(x_test)) > 0.5).numpy().astype(int).flatten()

visualize_datapoints(x_test * std + mean, preds)


In [ ]:
import torch
import torch.nn as nn


class MLP2(nn.Module):
  def __init__(self):
    super().__init__()

    self.layers = nn.Sequential(
            nn.Linear(2, 32),
            nn.Sigmoid(),
            nn.Linear(32, 16),
            nn.Sigmoid(),
            nn.Linear(16, 8),
            nn.Sigmoid(),
            nn.Linear(8, 4),
            nn.Sigmoid(),
            nn.Linear(4, 1),
            nn.Sigmoid()
    )

    for layer in self.layers:
        if isinstance(layer, nn.Linear):
            nn.init.xavier_uniform_(layer.weight)

  def forward(self, x):
    return self.layers(x)


In [ ]:
mean = X.mean(axis = 0)
std = X.std(axis = 0)
x_norm = (X - mean) / std


x_train, x_test = x_norm[:50000], x_norm[50000:]
y_train, y_test = Y[:50000], Y[50000:]


X_train = torch.tensor(x_train, dtype=torch.float32)
Y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(x_test, dtype=torch.float32)
Y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

seeds = [0, 1, 2, 3, 4]
results = []


train_accuracy, test_accuracy = [], []

for seed in seeds:
  torch.manual_seed(seed)
  model = MLP2()

  losses, train_accs, test_accs = train(
        model,
        X_train, Y_train,
        X_test, Y_test,
        nn.BCELoss(),
        accuracy,
        batch_size=128,
        max_epochs=500,
        lr=0.001
    )
  results.append((losses, train_accs, test_accs))

plt.figure(figsize=(12,5))
for i, (losses, train_accs, test_accs) in enumerate(results):
  plt.subplot(1,2,1)
  plt.plot(losses, label=f'Seed {i}')
  plt.title('Training Loss')
  plt.subplot(1,2,2)
  plt.plot(test_accs, label=f'Seed {i}')
  plt.title('Test Accuracy')
plt.legend()
plt.show()


with torch.no_grad():
    preds = (model(torch.FloatTensor(x_test)) > 0.5).numpy().astype(int).flatten()

visualize_datapoints(x_test * std + mean, preds)  # rescale back to original coords


In [ ]:
final_train_accs = [r[1][-1] for r in results]
final_test_accs = [r[2][-1] for r in results]

print("\n" + "="*60)
print("FINAL RESULTS - Problem 1c")
print("="*60)
print(f"Mean Train Accuracy: {np.mean(final_train_accs):.4f} ± {np.std(final_train_accs):.4f}")
print(f"Mean Test Accuracy:  {np.mean(final_test_accs):.4f} ± {np.std(final_test_accs):.4f}")
print("="*60)

print("\nIndividual Seed Results:")
for i, (train_acc, test_acc) in enumerate(zip(final_train_accs, final_test_accs)):
    print(f"Seed {i}: Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}")

## Problem 2

All code for Problem 2 should go below. We provide data loaders and relevant imports to get you started. If you are working locally (instead of using Google Colab), we recommend using Conda to install pytorch (https://pytorch.org).

### Imports

In [ ]:
## Additional External Libraries (Deep Learning)
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as tfs
from PIL import Image
from torchvision.datasets import FashionMNIST
from torch.utils.data import SubsetRandomSampler

### Data Loading

In [ ]:
# Hyperparameter (Feel Free to Change These, but Make Sure your Training Loop Still Works as Expected)
TRAIN_BATCH_SIZE = 50
VAL_BATCH_SIZE = 50
TEST_BATCH_SIZE = 1

# Transform data to PIL images
transforms = tfs.Compose([tfs.ToTensor()])

# Train/Val Subsets
train_mask = range(50000)
val_mask = range(50000, 60000)

# Download/Load Dataset
train_dataset = FashionMNIST('./data', train=True, transform=transforms, download=True)
test_dataset = FashionMNIST('./data', train=False, transform=transforms, download=True)

# Data Loaders
train_dataloader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, sampler=SubsetRandomSampler(train_mask))
val_dataloader = DataLoader(train_dataset, batch_size=VAL_BATCH_SIZE, sampler=SubsetRandomSampler(val_mask))
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE)

## Problem 2a)

### Design Model

In [ ]:
class CNNet(nn.Module):

    """

    """

    def __init__(self):
        """

        """
        ## Inherent Torch Module
        super(CNNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 3 * 3, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        """

        """
        x = self.conv_layers(x)
        x = x.flatten(start_dim = 1)
        x = self.fc_layers(x)
        return x

### Model Training

In [ ]:
def train(model,
          optimizer,
          loss,
          lr,
          epochs=50,
          train_dataloader=train_dataloader,
          val_dataloader=val_dataloader,
          test_dataloader=test_dataloader,
          **kwargs):


  model = model.cuda()

  train_losses, train_accuracies, val_accuracies = [], [], []
  best_validation_acc = 0
  best_model = None


  for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        t_correct = 0
        t_total = 0

        for inputs, labels in train_dataloader:
            inputs = inputs.cuda()
            labels = labels.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            l = loss(outputs, labels)
            l.backward()
            optimizer.step()

            running_loss += l.item()
            _, predicted = torch.max(outputs.data, 1)
            t_total += labels.size(0)
            t_correct += (predicted == labels).sum().item()

        avg_loss = running_loss / len(train_dataloader)
        train_acc = 100 * t_correct / t_total
        train_losses.append(avg_loss)
        train_accuracies.append(train_acc)

        # validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for val_inputs, val_labels in val_dataloader:
                val_inputs = val_inputs.cuda()
                val_labels = val_labels.cuda()

                outputs = model(val_inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_total += val_labels.size(0)
                val_correct += (predicted == val_labels).sum().item()

        val_acc = 100 * val_correct / val_total
        val_accuracies.append(val_acc)


        # saving best model
        if val_acc > best_validation_acc:
          best_validation_acc = val_acc
          best_model = model.state_dict().copy()

  model.load_state_dict(best_model)

  # training accuracy on best bodel

  model.eval()
  final_train_correct = 0
  final_train_total = 0
  with torch.no_grad():
    for train_inputs, train_labels in train_dataloader:
      train_inputs, train_labels = train_inputs.cuda(), train_labels.cuda()
      outputs = model(train_inputs)
      _, predicted = torch.max(outputs.data, 1)
      final_train_total += train_labels.size(0)
      final_train_correct += (predicted == train_labels).sum().item()

  final_train_acc = 100 * final_train_correct / final_train_total


  # testing
  model.eval()
  test_correct = 0
  test_total = 0

  with torch.no_grad():
    for test_inputs, test_labels in test_dataloader:
      test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
      outputs = model(test_inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += test_labels.size(0)
      test_correct += (predicted == test_labels).sum().item()

  test_acc = 100 * test_correct / test_total




  print(f'\n{"="*50}')
  print("FINAL RESULTS")
  print(f"{'='*50}")
  print(f"Training Accuracy: {final_train_acc:.2f}%")
  print(f"Validation Accuracy: {best_validation_acc:.2f}%")
  print(f"Test Accuracy: {test_acc:.2f}%")
  print(f"{'='*50}")

  plt.subplot(1, 2, 1)
  plt.plot(train_losses, 'b-', linewidth=2)
  plt.title('Training')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.grid(True, alpha=0.3)

  plt.subplot(1, 2, 2)
  plt.plot(val_accuracies, 'g-', linewidth=2, label='Validation Accuracy')
  plt.plot(train_accuracies, 'r--', alpha=0.7, label='Training Accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy (%)')
  plt.legend()
  plt.grid(True, alpha=0.3)

  plt.tight_layout()
  plt.show()


  return train_losses, train_accuracies, val_accuracies, final_train_acc, best_validation_acc, test_acc

In [ ]:
## Hyperparameters
EPOCH = 50
LR = 0.01

## Setting up the model, optimizer, and loss function
model = CNNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_f = nn.CrossEntropyLoss()

In [ ]:
## Run Training Loop
out = train(model, optimizer, loss_f, LR)



## Problem 2b)

Now try to improve your model using additional techniques learned during class. You should be able to use the same training function as above, but will need to create a new model architecture.

### Data Loading

You should maintain the splits from above, but feel free to alter the dataloaders (i.e. transforms) as you wish.

In [ ]:
# Hyperparameter (Feel Free to Change These, but Make Sure your Training Loop Still Works as Expected)
TRAIN_BATCH_SIZE = 50
VAL_BATCH_SIZE = 50
TEST_BATCH_SIZE = 1

# Transform data to PIL images
transforms = tfs.Compose([
    tfs.ToTensor(),
    tfs.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1] range
])

# Train/Val Subsets
train_mask = range(50000)
val_mask = range(50000, 60000)

# Download/Load Dataset
train_dataset = FashionMNIST('./data', train=True, transform=transforms, download=True)
test_dataset = FashionMNIST('./data', train=False, transform=transforms, download=True)

# Data Loaders
train_dataloader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, sampler=SubsetRandomSampler(train_mask))
val_dataloader = DataLoader(train_dataset, batch_size=VAL_BATCH_SIZE, sampler=SubsetRandomSampler(val_mask))
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE)

### Model Design

In [ ]:
class CNNImproved(nn.Module):
  def __init__(self):
    super(CNNImproved, self).__init__()
    self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

    self.fc_layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 10)
        )

  def forward(self, x):
        x = self.conv_layers(x)
        x = x.flatten(start_dim=1)
        x = self.fc_layers(x)
        return x


### Model Training

In [ ]:
model = CNNImproved()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_f = nn.CrossEntropyLoss()

out = train(model, optimizer, loss_f, LR)

## Problem 2c)

Write down your response in the final report.

I did a bit of research in Geoffery Hilton's Distilling the Knowledge in a Neural Netowrk (2015). This technique is as follows. A large neural network (the teacher) is trained on the FashionMNIST dataset to achieve a high accuracy. Its outputs are stored as the "knowledge" and soften using temperature T > 1, a small change to the softmax function. Then, a small network (the student) is with fewer layers is trained. However, instead of learning from the true dataset, it uses the soften outputs from the teacher. This technique allows for the student to be a much smaller neural network, but can achieve high accuracy because it leverages the teacher's knowledge.
